In [ ]:
!pwd

In [7]:
import torch
import torch.nn as nn

class CosineAnnealingLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
        self,
        optimizer: torch.optim.Optimizer,
        warmup_epochs: int,
        max_epochs: int,
        warmup_start_lr: float = 0.00001,
        eta_min: float = 0.00001,
        last_epoch: int = -1,
    ):
        """
        Args:
            optimizer (torch.optim.Optimizer):
                最適化手法インスタンス
            warmup_epochs (int):
                linear warmupを行うepoch数
            max_epochs (int):
                cosine曲線の終了に用いる 学習のepoch数
            warmup_start_lr (float):
                linear warmup 0 epoch目の学習率
            eta_min (float):
                cosine曲線の下限
            last_epoch (int):
                cosine曲線の位相オフセット
        学習率をmax_epochsに至るまでコサイン曲線に沿ってスケジュールする
        epoch 0からwarmup_epochsまでの学習曲線は線形warmupがかかる
        https://pytorch-lightning-bolts.readthedocs.io/en/stable/schedulers/warmup_cosine_annealing.html
        """
        self.warmup_epochs = warmup_epochs
        self.max_epochs = max_epochs
        self.warmup_start_lr = warmup_start_lr
        self.eta_min = eta_min
        super().__init__(optimizer, last_epoch)
        return None

    def get_lr(self):
        if self.last_epoch == 0:
            return [self.warmup_start_lr] * len(self.base_lrs)
        if self.last_epoch < self.warmup_epochs:
            return [
                group["lr"] + (base_lr - self.warmup_start_lr) / (self.warmup_epochs - 1)
                for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups)
            ]
        if self.last_epoch == self.warmup_epochs:
            return self.base_lrs
        if (self.last_epoch - 1 - self.max_epochs) % (2 * (self.max_epochs - self.warmup_epochs)) == 0:
            return [
                group["lr"] + (base_lr - self.eta_min) * (1 - math.cos(math.pi / (self.max_epochs - self.warmup_epochs))) / 2
                for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups)
            ]

        return [
            (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs) / (self.max_epochs - self.warmup_epochs)))
            / (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs - 1) / (self.max_epochs - self.warmup_epochs)))
            * (group["lr"] - self.eta_min)
            + self.eta_min
            for group in self.optimizer.param_groups
        ]

In [8]:
model = nn.Linear(1,1)
max_epoch = 128  # 学習終了のepoch数は最初から与える
iter_step = 4  # (ダミー)ミニバッチで学習する場合のイテレーションステップ数=バッチサイズ/ミニバッチサイズ

optimizer = torch.optim.AdamW(  
    model.parameters(), 
    lr=0.001,
    weight_decay=0.02)

lr_scheduler = CosineAnnealingLR(
    optimizer,
    max_epochs=max_epoch,
    warmup_epochs=8,
    warmup_start_lr=0.0001,
    eta_min=0.00001)

In [9]:
curves = []
for e in range(max_epoch):
    for s in range(iter_step):
        # 各イテレーションでパラメータを更新
        optimizer.step()
        curves += [optimizer.param_groups[0]["lr"]]
    # 各epoch終了後にスケジューラで最適化学習率を更新
    lr_scheduler.step()

plt.plot(curves)
plt.savefig("lr_curves.png")

NameError: name 'math' is not defined

In [15]:

arch = 'resnet18'
dim_dict = {
    'resnet18':512,
}
dim = dim_dict.get(arch, 1280)

print(dim)

512


In [22]:
from PIL import Image
import matplotlib.pyplot as plt
import os

image_path = '/root/Development/Kaggle/ISIC2024/data/raw/train-image/image/'
# image_path = 'Development/Kaggle/ISIC2024/main/models'

image_list = os.listdir(image_path)
image_list_10 = image_list[:10]

print(image_list_10)

# Open the image file
for path in image_list_10:
    image = Image.open(image_path + path)
    print(image.size)

    # # Display the image
    # plt.imshow(image)
    # plt.axis('off')  # Hide the axis
    # plt.show()


['ISIC_5726301.jpg', 'ISIC_4336258.jpg', 'ISIC_1510555.jpg', 'ISIC_0428056.jpg', 'ISIC_8108962.jpg', 'ISIC_8143544.jpg', 'ISIC_2473523.jpg', 'ISIC_1672562.jpg', 'ISIC_9741000.jpg', 'ISIC_3008170.jpg']
(137, 137)
(131, 131)
(109, 109)
(121, 121)
(137, 137)
(117, 117)
(127, 127)
(169, 169)
(113, 113)
(141, 141)


In [3]:
import torch
import timm

# Load the resnet18 model
model = timm.create_model('resnet18', pretrained=False, num_classes=0)

# Print the model architecture
print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [7]:
import numpy as np

logit=0
for i in range(1):
    logit += np.random.randn(10,1)
logit = logit/len('a')

print(logit)

[[ 0.54835175]
 [ 0.21671934]
 [ 0.17882353]
 [-1.88504006]
 [-0.0975358 ]
 [ 1.46056303]
 [ 1.07305814]
 [ 1.09843883]
 [ 0.94972329]
 [-0.94186006]]


In [2]:
import torch
t = torch.randn(4,4)
print(t)
torch.special.expit(t)

tensor([[ 0.0266, -2.0024, -0.7541,  0.0875],
        [ 0.6679, -1.3453, -0.5650, -0.4380],
        [-1.6116, -0.8948, -0.0234,  2.8518],
        [ 0.0050,  0.4251,  0.2849, -1.4315]])


tensor([[0.5067, 0.1189, 0.3199, 0.5219],
        [0.6610, 0.2066, 0.3624, 0.3922],
        [0.1664, 0.2901, 0.4942, 0.9454],
        [0.5012, 0.6047, 0.5707, 0.1929]])

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt

def plot_scheduler(scheduler_class, optimizer_class, scheduler_params, optimizer_params, epochs, option_name=''):
    optimizer = optimizer_class([torch.tensor(0.0, requires_grad=True)], **optimizer_params)
    scheduler = scheduler_class(optimizer, **scheduler_params)

    lr_log = []
    for epoch in range(epochs):
        lr_log.append(optimizer.param_groups[0]['lr'])
        scheduler.step()

    plt.figure(figsize=(8, 6))
    plt.plot(range(epochs), lr_log)
    plt.title(f"{scheduler_class.__name__} with {optimizer_class.__name__}")
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.savefig(f'{scheduler_class.__name__}' + f'{option_name}' + '.png')
    plt.show()

# LambdaLR
plot_scheduler(lr_scheduler.LambdaLR, optim.Adam, {'lr_lambda': lambda epoch: 0.95 ** epoch}, {'lr': 0.1}, 100)

# StepLR
plot_scheduler(lr_scheduler.StepLR, optim.Adam, {'step_size': 10, 'gamma': 0.8}, {'lr': 0.1}, 100)

# MultiStepLR
plot_scheduler(lr_scheduler.MultiStepLR, optim.Adam, {'milestones': [30, 80], 'gamma': 0.1}, {'lr': 0.1}, 100)

# ExponentialLR
plot_scheduler(lr_scheduler.ExponentialLR, optim.Adam, {'gamma': 0.95}, {'lr': 0.1}, 100)

# CosineAnnealingLR
plot_scheduler(lr_scheduler.CosineAnnealingLR, optim.Adam, {'T_max': 20, 'eta_min': 0.001}, {'lr': 0.1}, 100)

# ReduceLROnPlateau
optimizer = optim.Adam([torch.tensor(0.0, requires_grad=True)], lr=0.1)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

lr_log = []
for epoch in range(100):
    lr_log.append(optimizer.param_groups[0]['lr'])
    scheduler.step(epoch)

plt.figure(figsize=(8, 6))
plt.plot(range(100), lr_log)
plt.title("ReduceLROnPlateau with Adam")
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.savefig('ReduceLROnPlateau.png')
plt.show()

# CyclicLR-triangular
plot_scheduler(lr_scheduler.CyclicLR, optim.SGD, {'base_lr': 0.001, 'max_lr': 0.1, 'step_size_up': 20, 'mode': 'triangular'}, {'lr': 0.1, 'momentum': 0.9}, 100, '-1')

# CyclicLR-triangular2
plot_scheduler(lr_scheduler.CyclicLR, optim.SGD, {'base_lr': 0.001, 'max_lr': 0.1, 'step_size_up': 20, 'mode': 'triangular2'}, {'lr': 0.1, 'momentum': 0.9}, 100, '-2')

# CyclicLR-exp_range
plot_scheduler(lr_scheduler.CyclicLR, optim.SGD, {'base_lr': 0.001, 'max_lr': 0.1, 'step_size_up': 20, 'mode': 'exp_range', 'gamma': 0.95}, {'lr': 0.1, 'momentum': 0.9}, 100, '-exp')

# OneCycleLR
plot_scheduler(lr_scheduler.OneCycleLR, optim.SGD, {'max_lr': 0.1, 'total_steps': 100, 'pct_start': 0.3, 'anneal_strategy': 'cos'}, {'lr': 0.1, 'momentum': 0.9}, 100)

# CosineAnnealingWarmRestarts
plot_scheduler(lr_scheduler.CosineAnnealingWarmRestarts, optim.Adam, {'T_0':10}, {'lr': 0.1}, 100)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits

# Load a sample dataset
digits = load_digits()
X = digits.data
y = digits.target

# Perform t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
X_tsne = tsne.fit_transform(X)

# Plotting the results
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='viridis')
plt.colorbar(scatter)
plt.title('t-SNE visualization of the Digits dataset')
plt.savefig('/root/Development/Kaggle/ISIC2024/reports/figures/t-SNE_digits.png')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits

# Load a sample dataset
digits = load_digits()
X = digits.data
y = digits.target

# Perform PCA for initialization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# t-SNE with PCA initialization
tsne_pca = TSNE(n_components=2, init='pca', perplexity=30, n_iter=300)
X_tsne_pca = tsne_pca.fit_transform(X)

# t-SNE with random initialization
tsne_random = TSNE(n_components=2, init='random', perplexity=30, n_iter=300)
X_tsne_random = tsne_random.fit_transform(X)

# Plotting the results
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
scatter_pca = plt.scatter(X_tsne_pca[:, 0], X_tsne_pca[:, 1], c=y, cmap='viridis')
plt.title('t-SNE with PCA Initialization')
plt.colorbar(scatter_pca)

plt.subplot(1, 2, 2)
scatter_random = plt.scatter(X_tsne_random[:, 0], X_tsne_random[:, 1], c=y, cmap='viridis')
plt.title('t-SNE with Random Initialization')
plt.colorbar(scatter_random)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import time

# Load a sample dataset
digits = load_digits()
X = digits.data
y = digits.target

# Perform PCA
start_time = time.time()
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
pca_time = time.time() - start_time

# Perform t-SNE with PCA initialization
start_time = time.time()
tsne = TSNE(n_components=2, init='pca', perplexity=30, n_iter=300)
X_tsne_pca = tsne.fit_transform(X)
tsne_time = time.time() - start_time

# Perform UMAP
start_time = time.time()
umap_reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = umap_reducer.fit_transform(X)
umap_time = time.time() - start_time

# Perform DensMAP (Densified Manifold Approximation and Projection)
start_time = time.time()
densmap_reducer = umap.UMAP(densmap=True, random_state=42)
X_densmap = densmap_reducer.fit_transform(X)
densmap_time = time.time() - start_time

# Plotting the results
plt.figure(figsize=(20, 10))

plt.subplot(2, 2, 1)
scatter_pca = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'PCA (time: {pca_time:.2f} s)')
plt.colorbar(scatter_pca)

plt.subplot(2, 2, 2)
scatter_tsne_pca = plt.scatter(X_tsne_pca[:, 0], X_tsne_pca[:, 1], c=y, cmap='viridis', s=5)
plt.title(f't-SNE (time: {tsne_time:.2f} s)')
plt.colorbar(scatter_tsne_pca)

plt.subplot(2, 2, 3)
scatter_umap = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'UMAP (time: {umap_time:.2f} s)')
plt.colorbar(scatter_umap)

plt.subplot(2, 2, 4)
scatter_densmap = plt.scatter(X_densmap[:, 0], X_densmap[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'DensMAP (time: {densmap_time:.2f} s)')
plt.colorbar(scatter_densmap)

plt.tight_layout()
plt.savefig('/root/Development/Kaggle/ISIC2024/reports/figures/compare_RD_methods')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import time

# Load a sample dataset
digits = load_digits()
X = digits.data
y = digits.target

# Perform PCA
start_time = time.time()
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
pca_time = time.time() - start_time

# Perform t-SNE with PCA initialization
start_time = time.time()
tsne = TSNE(n_components=2, init='pca', perplexity=30, n_iter=300)
X_tsne_pca = tsne.fit_transform(X)
tsne_time = time.time() - start_time

# Perform UMAP
start_time = time.time()
umap_reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = umap_reducer.fit_transform(X)
umap_time = time.time() - start_time

# Perform DensMAP (Densified Manifold Approximation and Projection)
start_time = time.time()
densmap_reducer = umap.UMAP(densmap=True, random_state=42)
X_densmap = densmap_reducer.fit_transform(X)
densmap_time = time.time() - start_time

# Plotting the results
plt.figure(figsize=(20, 12))

plt.subplot(3, 2, 1)
plt.imshow(digits.images[0], cmap='gray')
plt.title('Sample Image 1')
plt.colorbar()

plt.subplot(3, 2, 2)
plt.imshow(digits.images[1], cmap='gray')
plt.title('Sample Image 2')
plt.colorbar()

plt.subplot(3, 2, 3)
scatter_pca = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'PCA (time: {pca_time:.2f} s)')
plt.colorbar(scatter_pca)

plt.subplot(3, 2, 4)
scatter_tsne_pca = plt.scatter(X_tsne_pca[:, 0], X_tsne_pca[:, 1], c=y, cmap='viridis', s=5)
plt.title(f't-SNE with PCA Initialization (time: {tsne_time:.2f} s)')
plt.colorbar(scatter_tsne_pca)

plt.subplot(3, 2, 5)
scatter_umap = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'UMAP (time: {umap_time:.2f} s)')
plt.colorbar(scatter_umap)

plt.subplot(3, 2, 6)
scatter_densmap = plt.scatter(X_densmap[:, 0], X_densmap[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'DensMAP (time: {densmap_time:.2f} s)')
plt.colorbar(scatter_densmap)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_digits

# Load the dataset
digits = load_digits()
X = digits.data
y = digits.target
images = digits.images

# Display all the input patterns
plt.figure(figsize=(10, 8))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(images[i], cmap='gray')
    plt.title(f'Digit: {y[i]}')
    plt.axis('off')
plt.suptitle('Input Patterns')
plt.savefig('/root/Development/Kaggle/ISIC2024/reports/figures/inputs.png')
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import time

# Load a sample dataset
digits = load_digits()
X = digits.data
y = digits.target

# Perform PCA
start_time = time.time()
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
pca_time = time.time() - start_time

# Perform t-SNE with PCA initialization
start_time = time.time()
tsne = TSNE(n_components=2, init='pca', perplexity=30, n_iter=300)
X_tsne_pca = tsne.fit_transform(X)
tsne_time = time.time() - start_time

# Perform UMAP
start_time = time.time()
umap_reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = umap_reducer.fit_transform(X)
umap_time = time.time() - start_time

# Perform DensMAP (Densified Manifold Approximation and Projection)
start_time = time.time()
densmap_reducer = umap.UMAP(densmap=True, random_state=42)
X_densmap = densmap_reducer.fit_transform(X)
densmap_time = time.time() - start_time

# Plotting the results one by one
plt.figure()
scatter_pca = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'PCA (time: {pca_time:.2f} s)')
plt.colorbar(scatter_pca)
plt.show()

plt.figure()
scatter_tsne_pca = plt.scatter(X_tsne_pca[:, 0], X_tsne_pca[:, 1], c=y, cmap='viridis', s=5)
plt.title(f't-SNE (time: {tsne_time:.2f} s)')
plt.colorbar(scatter_tsne_pca)
plt.show()

plt.figure()
scatter_umap = plt.scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'UMAP (time: {umap_time:.2f} s)')
plt.colorbar(scatter_umap)
plt.show()

plt.figure()
scatter_densmap = plt.scatter(X_densmap[:, 0], X_densmap[:, 1], c=y, cmap='viridis', s=5)
plt.title(f'DensMAP (time: {densmap_time:.2f} s)')
plt.colorbar(scatter_densmap)
plt.show()
